# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('homepage_actions.csv')
print(len(df))
print(len(df.id.unique()))

df.action.value_counts()

#no of unique IDs: 6328
#no pageviews: 6328
#no of clicks: 1860

id_click = set(df[df.action=='click']['id'].unique())
id_views = set(df[df.action=='view']['id'].unique())

id_click_and_view = id_click.intersection(id_views)
click_without_view = id_click.difference(id_views)
view_no_click = id_views.difference(id_click)
len(click_without_view) # there are no ID's that clicked but didn't view
len(view_no_click) # there are 4468 ID's that viewed but did not click

8188
6328


4468

In [26]:
# Is there any overlap between the control and experiment groups? 

control = set(df[df.group=='control']['id'].unique())
experimental = set(df[df.group=='experiment']['id'].unique())

group_intersection = control.intersection(experimental)
group_intersection

#there is no intersection between control and experimental group

set()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [39]:
control = df.loc[df['group']=='control']
control_click = control.loc[control['action']=='click']


exp = df.loc[df['group']=='experiment']
exp_click = exp.loc[exp['action']=='click']

len_control = len(control)
clicks_control = len(control_click)

len_exp = len(exp)
clicks_exp = len(exp_click)

control_success = clicks_control/len_control
exp_success = clicks_exp/len_exp

print(control_success,exp_success)


0.21857410881801126 0.23649337410805302


4264

In [42]:
print("Sample sizes:\tControl: {}\tExperiment: {}".format(len_control, len_exp))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(clicks_control,clicks_exp ))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control_success,exp_success))
control.head()

Sample sizes:	Control: 4264	Experiment: 3924
Total Clicks:	Control: 932	Experiment: 928
Average click rate:	Control: 0.21857410881801126	Experiment: 0.23649337410805302


,timestamp,id,group,action
3,2016-09-24 19:59:02.646620,671993,control,view
9,2016-09-24 21:00:12.278374,560027,control,view
23,2016-09-25 00:25:14.141290,281985,control,view
24,2016-09-25 01:14:48.603202,407864,control,view
25,2016-09-25 02:16:11.046654,342984,control,view


In [45]:
#USING BINARY COLUMNS

#Convert clicks into a binary variable on a user-by-user-basis

df['count'] = 1

control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

In [49]:
control.head()

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [50]:
import flatiron_stats as fs

fs.p_value_welch_ttest(control.click, experiment.click)

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [53]:
control_mean = control.click.mean()

expected_experiment_clicks_under_null = control_mean * len(experiment)
print(expected_experiment_clicks_under_null)


838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [ ]:
#Your code here

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
#Your code here

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.